In [1]:
import pyspark 
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/29 23:23:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df_green = spark.read.parquet('data/pq/green/*/*')

25/10/29 23:29:04 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: data/pq/green/*/*.
java.io.FileNotFoundException: File data/pq/green/*/* does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:917)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:1238)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:907)
	at org.apache.hadoop.fs.FilterFileSystem.getFileStatus(FilterFileSystem.java:462)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:56)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:381)
	at org.apache.spark.sql.catalyst.analysis.ResolveDataSource.org$apache$spark$sql$catalyst$analysis$ResolveDataSource$$loadV1BatchSource(ResolveDataSource.scala:143)
	at org.apache.spark.sql.catalyst.analysis.ResolveDataSource

In [6]:
df_green.registerTempTable('green')

/home/binnassir/anaconda3/lib/python3.13/site-packages/pyspark/sql/classic/dataframe.py:178: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [17]:
df_green_revenue = spark.sql(""" 
 SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour,
    PULocationID AS zone, 
    SUM(total_amount) AS amount,
    COUNT(1) AS number_records


    FROM 
      green
    WHERE
      lpep_pickup_datetime >= '2020-01-01 00:00:00'
    GROUP BY
      1,2
""")

In [24]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green',mode='overwrite')

In [16]:
spark.sparkContext.uiWebUrl

'http://de-zoomcamp-week5.me-central1-b.c.zoomcamp-taxi-ny.internal:4040'

In [19]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.registerTempTable('yellow')

25/10/30 00:00:35 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: data/pq/yellow/*/*.
java.io.FileNotFoundException: File data/pq/yellow/*/* does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:917)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:1238)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:907)
	at org.apache.hadoop.fs.FilterFileSystem.getFileStatus(FilterFileSystem.java:462)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:56)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:381)
	at org.apache.spark.sql.catalyst.analysis.ResolveDataSource.org$apache$spark$sql$catalyst$analysis$ResolveDataSource$$loadV1BatchSource(ResolveDataSource.scala:143)
	at org.apache.spark.sql.catalyst.analysis.ResolveDataSour

In [21]:
df_yellow_revenue = spark.sql(""" 
 SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour,
    PULocationID AS zone, 
    SUM(total_amount) AS amount,
    COUNT(1) AS number_records


    FROM 
      yellow
    WHERE
      tpep_pickup_datetime >= '2020-01-01 00:00:00'
    GROUP BY
      1,2
""")

In [25]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow',mode='overwrite')